In [346]:
import numpy as np
import pandas as pd
import pickle
import numbers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from tkinter import *

In [3]:
df = pickle.load(open('houses.p','rb'))

In [4]:
df.head()

,buurt_code,V1.x,endprice,url,realtor,buurt_naam,wijk_code,bathroom.ligbad,bathroom.douche,bathroom.toilet,...,startDay,startDayofweek,startDayofyear,startIs_month_end,startIs_month_start,startIs_quarter_end,startIs_quarter_start,startIs_year_end,startIs_year_start,startElapsed
57585,BU07980000,48990,12.464583,/koop/verkocht/hilvarenbeek/appartement-473706...,/makelaars/tilburg/19266-lupker-makelaardij/,Hilvarenbeek,WK079800,1,1,0,...,27.0,4.0,27.0,False,False,False,False,False,False,2293.0
57584,BU07980000,48989,12.549662,/koop/verkocht/hilvarenbeek/appartement-473706...,/makelaars/tilburg/19266-lupker-makelaardij/,Hilvarenbeek,WK079800,1,0,1,...,27.0,4.0,27.0,False,False,False,False,False,False,2293.0
50443,BU06870000,42289,12.205573,/koop/verkocht/middelburg/appartement-47228274...,/makelaars/goes/20112-kuub-makelaars/,Binnenstad,WK068700,0,1,0,...,26.0,5.0,330.0,False,False,False,False,False,False,2231.0
74786,BU18830502,59549,12.037654,/koop/verkocht/geleen/huis-84185703-slotmakers...,/makelaars/munstergeleen/62973-quaden-makelaar...,Geleen-Noord,WK188305,0,0,0,...,13.0,3.0,164.0,False,False,False,False,False,False,2796.0
57583,BU07980000,48988,12.896717,/koop/verkocht/hilvarenbeek/appartement-473706...,/makelaars/tilburg/19266-lupker-makelaardij/,Hilvarenbeek,WK079800,1,2,0,...,27.0,4.0,27.0,False,False,False,False,False,False,2293.0


In [209]:
df['lotsurface'] = df['lotsurface'].fillna(-1)

In [348]:
isinstance('a', numbers.Number)

False

In [370]:
# weights --> dictionary of feature name to importance
# data --> pandas datafram of houses with all relevant features
# rcf --> list of all relevant features 
# puf --> list of all potentially upgradable features -should be a subset of relevant features 
# thres --> maximum distance a house can have from the target house if it is to be put in a reference set 
# target_house

def normalize_column(x):
    min_max_scaler = MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return x_scaled, min_max_scaler

def weightedL2(a,b,w):
    q = a-b
    return np.sqrt((w*q*q).sum())

# rcf --> relevent comparable features rcfv--> relevant comparable feature values puf --> potential upgradable features
def get_reference_indices(weights, data, thres, target_values):
    reference_indices = []
    for i, house in enumerate(data):
#         print(weightedL2(target_values, house,weights))
        if weightedL2(target_values, house,weights) <  thres:
#             print(weightedL2(target_values, house,weights))
            reference_indices.append(i)
    return reference_indices

def get_reference_sets(weights, rcf, ccf, data, target_house, thres=1):
    df_ans = data.copy()
    df = data[rcf+ccf] # We only look only at the columns that were determined to be relevant for comparison
    t_house = target_house[rcf+ccf]
    for col in ccf:
        val = t_house[col].iloc[0]
        df = df[df[col]==val]
    for col in df[rcf].columns:
        if (df[col].dtype in [bool,float,int,np.int64]):
            a, b = normalize_column(df[[col]].values.astype(float))
            df[col] = a
            t_house[col] = b.transform(t_house[[col]].values.astype(float))
    target_values = t_house[rcf].values
    data_array = df[rcf].values
#     return target_values
    weights_arr = np.array(list(weights.copy().values()))
    reference_index = get_reference_indices(weights_arr, data_array, thres, target_values)
    return df_ans.iloc[reference_index]

In [386]:
weights = {"volume" : 0.275818,
            "livingspace" : 0.251524,
            "Gemiddelde woningwaarde:x 1 000 euro" : 0.113488,
#            "housetype" : 0.054045,
           "V1.x":0.036266,
           "lotsurface":0.029131,
           "yearofconstruction":0.020824,
           "longitude":0.017308,
           'latitude':0.012521}
#            'housesubtype':0.011934}

#Normalize weights 
sum_vals = sum(weights.values())
for key in weights.keys():
    weights[key] = weights[key] / sum_vals
    
rcf = list(weights.keys())
ccf = ["housesubtype","housetype"]
thresh = 0.1
t_house = pd.DataFrame(df.iloc[0]).T

In [387]:
result_1 = get_reference_sets(weights=weights, rcf = rcf, ccf = ccf, data= df, target_house= t_house,thres=thresh)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [388]:
len(result_1)

2416

In [385]:
# User input 
Volume = float( input("Enter House Volume in m3 : "))
Living_space = float (input("Enter living space in m2 : "))
House_type = input("Please enter the type of the house : ")
Postal_code = input('Please enter a 4 digit postal code : ')
Lot_surface = float( input('Please entere the lot surface in m2 : '))
Year_consrt = int( input("Please enter year of construction : "))
Long = float( input("Please enter longtitude : "))
Lang = float( input("Please enter Langtitude : "))
House_sub_type = input("Please enter house sub type : ")

Enter House Volume in m3 : ξ


ValueError: could not convert string to float: 'ξ'

In [48]:
df.housesubtype.dtype

CategoricalDtype(categories=['2-onder-1-kapwoning', 'beneden + bovenwoning',
                  'benedenwoning', 'bovenwoning', 'dubbel benedenhuis',
                  'eindwoning', 'galerijflat',
                  'geschakelde 2-onder-1-kapwoning', 'geschakelde woning',
                  'halfvrijstaande woning', 'hoekwoning', 'maisonnette',
                  'penthouse', 'portiekflat', 'portiekwoning',
                  'studentenkamer', 'tussenverdieping', 'tussenwoning',
                  'verspringend', 'vrijstaande woning'],
                 ordered=True)